# 🚀 HUFS-OOS-cs-student-ragbot: EC2 + Docker + GitHub Actions 배포 가이드

이 문서는 **HUFS-OOS-cs-student-ragbot** 프로젝트를 AWS EC2에 **Docker**로 패키징하고, **GitHub Actions(CI/CD)** 를 통해 자동 배포하며, **DuckDNS**로 도메인 연결까지 완료하는 전체 과정을 정리한 가이드입니다.

---

## 📦 프로젝트 요약

- **서버 배포 대상:** AWS EC2 Ubuntu 22.04 LTS
- **도메인 연결:** DuckDNS (`http://hufscomchatbot.duckdns.org`)
- **퍼블릭 IP:** `13.209.135.41`
- **SSH Key 파일:** `hufschatbot.pem`
- **백엔드 서버 포트:** `8000`

---

## ✅ 1. EC2 인스턴스 생성 및 접속

1. **AWS EC2** 대시보드 → Ubuntu 22.04 LTS 인스턴스 생성
2. **보안 그룹** 설정:
   - SSH: 포트 22
   - HTTP: 포트 80
   - FastAPI: 포트 8000
3. **탄력적 IP** 할당 후 연결 (`13.209.135.41`)
4. `.pem` 키파일 다운로드 (`hufschatbot.pem`)

### 🔐 SSH 접속

```bash
chmod 400 hufschatbot.pem
ssh -i "hufschatbot.pem" ubuntu@13.209.135.41
```

---

## ✅ 2. 서버 환경 구성

```bash
sudo apt update && sudo apt upgrade -y
sudo apt install docker.io nginx -y
sudo systemctl enable docker
sudo systemctl start docker
```

---

## ✅ 3. GitHub Secrets 설정

GitHub 저장소 > Settings > Secrets and variables > Actions 메뉴에서 다음 4가지를 등록합니다.

- EC2_HOST: 13.209.135.41
- EC2_USER: ubuntu
- EC2_KEY: hufschatbot.pem 파일을 base64로 인코딩한 값 (base64 -i hufschatbot.pem)
- ENV_FILE: .env 파일 전체 문자열 (OPENAI_API_KEY)

---

## ✅ 4. 프로젝트에 Dockerfile 작성

Dockerfile:

```Dockerfile
FROM python:3.10-slim

WORKDIR /app

COPY requirements.txt ./
RUN pip install --no-cache-dir -r requirements.txt

COPY . .

EXPOSE 8000

CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]
```

---

## ✅ 5. GitHub Actions CI/CD 설정

.github/workflows/deploy.yml:

```yaml
name: Deploy to EC2

on:
  push:
    branches:
      - main

jobs:
  deploy:
    runs-on: ubuntu-latest

    steps:
    - name: Checkout
      uses: actions/checkout@v3

    - name: Decode SSH Key
      run: |
        echo "${{ secrets.EC2_KEY }}" | base64 -d > key.pem
        chmod 400 key.pem

    - name: Copy files to EC2
      run: |
        scp -o StrictHostKeyChecking=no -i key.pem -r . ${{ secrets.EC2_USER }}@${{ secrets.EC2_HOST }}:/home/${{ secrets.EC2_USER }}/ragbot

    - name: Run Docker on EC2
      run: |
        ssh -o StrictHostKeyChecking=no -i key.pem ${{ secrets.EC2_USER }}@${{ secrets.EC2_HOST }} << 'EOF'
          cd ragbot
          echo "${{ secrets.ENV_FILE }}" > .env
          docker stop ragbot || true
          docker rm ragbot || true
          docker build -t ragbot .
          docker run -d --name ragbot -p 8000:8000 --env-file .env ragbot
        EOF
```

---

## ✅ 6. DuckDNS 연결

1.	https://www.duckdns.org 에서 도메인 생성 (hufscomchatbot)
2.	EC2에서 DuckDNS 스크립트 생성:

```bash
mkdir ~/duckdns
cd ~/duckdns
nano duck.sh
```

```bash
# duck.sh 내용
echo "url=https://www.duckdns.org/update?domains=hufscomchatbot&token=<YOUR_TOKEN>&ip=" | curl -k -o ~/duckdns/duck.log -K -
```

```bash
chmod 700 duck.sh
crontab -e
```

```cron
*/5 * * * * ~/duckdns/duck.sh >/dev/null 2>&1
```

---